# 教育反馈数据分析 - 数据清洗

这个 notebook 用于清洗教育反馈数据，为后续分析做准备。

In [ ]:
# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 设置显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

# 设置图表样式
plt.style.use('seaborn-v0_8')
%matplotlib inline

## 1. 数据加载

In [ ]:
# 加载数据
file_path = '../../data/raw/比赛数据.csv'
df = pd.read_csv(file_path, encoding='gbk')

# 显示数据的基本信息
print("数据集形状:", df.shape)
print("\n数据集列名:")
print(df.columns.tolist())
print("\n数据集前5行:")
display(df.head())

In [ ]:
# 检查数据类型和缺失值
print("数据类型:")
print(df.dtypes)
print("\n缺失值统计:")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

## 2. 数据清洗步骤

### 2.1 清理列名

首先，我们发现列名中有很多成对出现的列，前一个列的值是文字评价，后一个标题后面有数字2的列的值是数字评价，另外有很多列是全空列，没有意义，需要进行清理。

In [ ]:
# 1. 识别需要处理的列对
#    - 创建一个列表来存储将被删除的“文字评价”列名。
#    - 创建一个字典来存放“数字评价”列的重命名规则（{旧列名: 新列名}）。
columns_to_drop = []
rename_map = {}

# 为了高效查找，我们先将所有列名放入一个集合中
all_columns = set(df.columns)

# 遍历 DataFrame 的所有列名
for col_name in df.columns:
    # 检查当前列名是否以 '2' 结尾。这是判断数字评价列的关键。
    if col_name.endswith('2'):
        # 构造对应的、潜在的“文字评价”列名（即去掉末尾的 '2'）
        potential_text_col = col_name[:-1]

        # 检查这个潜在的“文字评价”列名是否真的存在于 DataFrame 的列中
        if potential_text_col in all_columns:
            # 如果存在，说明我们找到了一对符合“文字评价”和“数字评价（带'2'）”标准的列

            # 我们需要删除原始的“文字评价”列，所以将 `potential_text_col` 加入待删除列表
            columns_to_drop.append(potential_text_col)

            # 我们需要将“数字评价”列 (`col_name`) 重命名为不带 '2' 的名字 (`potential_text_col`)
            # 构建重命名映射：{当前列名: 目标新列名}
            rename_map[col_name] = potential_text_col

# --- 执行重命名和删除操作 ---

# 只有在找到了需要处理的列对时才执行操作
if rename_map:
    print(f"成功识别到 {len(rename_map)} 对需要处理的列。开始执行操作...")

    # 2. 执行列删除
    #    - 删除所有识别到的“文字评价”列。
    #    - columns_to_drop 列表中存储的就是这些原始的文字评价列的列名。
    #    - inplace=True 表示直接在原 DataFrame 上修改。
    df.drop(columns=columns_to_drop, inplace=True)

    # 3. 执行列重命名
    #    - 将所有识别到的“数字评价”列（带'2'）重命名为不带'2'的名字。
    #    - inplace=True 表示直接在原 DataFrame 上修改。
    df.rename(columns=rename_map, inplace=True)



    print("批量处理完成！")
else:
    print("未在 DataFrame 中找到符合条件的成对列（即没有列名以 '2' 结尾，且对应的原始列名也存在的配对）。")

# 处理空列
df.dropna(axis=1, how='all', inplace=True)

# 因为提交序号，提交时间和答题时长没有任何的参考价值所以这里也一并删除
columns_to_drop_specific = ["提交序号", "提交时间", "答题时长"]
existing_columns_to_drop = [col for col in columns_to_drop_specific if col in df.columns]
if existing_columns_to_drop:
    print(f"正在删除以下列: {existing_columns_to_drop}")
    df.drop(columns=existing_columns_to_drop, inplace=True)
    print("指定列已成功删除。")
else:
    print("您尝试删除的列 ('提交序号', '提交时间', '答题时长') 在 DataFrame 中均不存在。")

print("\n--- 处理后 DataFrame 的列名 ---")
print(df.columns.tolist())

# 打印处理后的 DataFrame 的前几行，以便您验证结果
print("\n--- 处理后 DataFrame 示例 ---")
print(df.head())
print("数据集形状:", df.shape)

发现有两列列名分别为"请您评价本校服务及教育教学是否符合以下情况（这道题选择“比较符合”即可，请不要选择其他项）"和“您评价本校实习实践各方面的满意度(这道题选择“比较满意”即可，请不要选择其他项)”，这两列得分不为80的应该全都可以认为是无效样本，删除对应行后，删除这两列，完成无效样本的去除

In [ ]:
# 定义目标列名
target_cols = [
    "请您评价本校服务及教育教学是否符合以下情况（这道题选择“比较符合”即可，请不要选择其他项）",
    "请您评价本校实习实践各方面的满意度（这道题选择“比较满意”即可，请不要选择其他项）"
]
# 记录初始行数
original_rows = len(df)
# 遍历处理每一列
for target_col in target_cols:
    if target_col in df.columns:
        print(f"\n正在处理列: '{target_col}'")

        # 1. 删除得分不为80的行（保留有效样本）
        df = df[df[target_col] == 80].copy()  # 使用copy()避免链式索引警告
        removed_rows = original_rows - len(df)
        print(f"已删除 {removed_rows} 个无效样本（得分不为80的行）")

        # 2. 删除该列本身
        df.drop(columns=[target_col], inplace=True)
        print(f"列 '{target_col}' 已成功删除")

        # 更新原始行数以供下一列处理
        original_rows = len(df)
    else:
        print(f"警告: 未找到列 '{target_col}'，跳过该列处理")

# 最终处理结果
print("\n--- 最终处理结果 ---")
print(f"剩余有效样本数: {len(df)}")
print(f"剩余列数: {len(df.columns)}")
print("数据集形状:", df.shape)
print("\n前5行数据预览:")
print(df.head())
columns_to_drop = [col for col in columns_to_drop if col not in target_cols]

发现列”您与任课教师课下交流的平均频度“仍然为文字，根据每周一次取1/7，每月一次取1/30，每学期一次取1/150，每年一次取1/365，对文字进行数据化


In [ ]:
# 定义频率映射规则
frequency_mapping = {
    '每周至少一次': 1/7,
    '每月至少一次': 1/30,
    '每学期至少一次': 1/150,
    '每年至少一次': 1/365,

}

# 指定需要转换的列名
frequency_col = "您与任课教师课下交流的平均频度"

# 检查该列是否存在
if frequency_col in df.columns:
    print(f"正在将列 '{frequency_col}' 从文字描述转换为数值...")

    # 使用 map() 方法进行转换
    df[frequency_col] = df[frequency_col].map(frequency_mapping)

    # 检查是否有未映射的值
    if df[frequency_col].isna().any():
        unmapped_values = df[frequency_col][df[frequency_col].isna()].unique()
        print(f"警告: 发现未映射的值: {unmapped_values}. 这些值将被保留为 NaN.")

    print("转换完成！")
else:
    print(f"列 '{frequency_col}' 不存在于 DataFrame 中。")

# 打印处理后的结果示例
if frequency_col in df.columns:
    print("\n--- 转换后的列示例 ---")
    print(df[[frequency_col]].head())
columns_to_drop.append(frequency_col)

### 接下来这些数值评分做缩放（到0-1尺度）

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 实例化一个scaler
scaler = MinMaxScaler(feature_range=(0, 1))

for col in columns_to_drop:
    df[col] = scaler.fit_transform(df[[col]]) # scaler期望一个二维数据

print(df[columns_to_drop])

In [ ]:
# 查看原始列名
print("原始列名数量:", len(df.columns))
for i, col in enumerate(df.columns[:]):
    print(f"{i+1:2d}. {col}")

In [ ]:
# 由于很多列名称实在是太长，所以我们要建立一个列名映射来精简数据列名

# 创建列名映射字典
column_mapping = {
    '您目前所在的专业': '学院',
    '您目前所在的专业.1': '专业',
    '您目前所在的年级是': '年级',
    '您的性别是': '性别',
    '您的政治面貌是': '政治面貌',
    # 学习情况相关
    '本学年，您的学习情况如何（课前运用网络课程平台、教材或资料进行自主预学，梳理知识并发现问题）': '课前预学',
    '本学年，您的学习情况如何（课堂上能跟上老师的讲解，主动提问或参与讨论，每节课学有所获）': '课堂参与',
    '本学年，您的学习情况如何（课后复习笔记及总结课堂教学相关知识）': '课后复习',
    '本学年，您的学习情况如何（除了阅读教材，还延伸阅读教师推荐的其他相关书目）': '延伸阅读',
    '完成作业': '完成作业时间',
    '除作业外的自习（如预习、复习、资料阅读）': '自习时间',
    '阅读专业相关的课外书籍': '课外阅读时间',
    '花费在网络课程平台上的时间': '网络课程时间',
    '做实验/科研项目': '实验科研时间',
    '参与专业相关社团活动': '社团活动时间',
    '参与学科竞赛活动': '竞赛活动时间',
    '参与其他学习活动（如升学）': '其他学习时间',
    # 学习互动情况
    '本学年，您和其他同学的学习互动情况如何（和同学合作完成课程任务或作业）': '同学合作',
    '本学年，您和其他同学的学习互动情况如何（和同学一起参与老师科研团队）': '参与科研团队',
    '本学年，您和其他同学的学习互动情况如何（和同学一起参加学科竞赛）': '参与学科竞赛',
    '本学年，您和其他同学的学习互动情况如何（从同学那里学习、吸收好的学习方法）': '学习同学方法',
    '您与任课教师课下交流的平均频度': '师生交流频度',
    # 思想政治课满意度
    '您对本校思想政治课的满意度（总体满意度）': '思政课总体满意度',
    '您对本校思想政治课的满意度（课程设置）': '思政课设置满意度',
    '您对本校思想政治课的满意度（教学内容）': '思政课内容满意度',
    '您对本校思想政治课的满意度（教学质量）': '思政课质量满意度',
    '您对本校思想政治课的满意度（教学效果）': '思政课效果满意度',
    # 专业课评价
    '请您评价本学年所修专业课在以下方面的情况（课程将知识、能力、素养进行了有机融合）': '专业课知识融合',
    '请您评价本学年所修专业课在以下方面的情况（课程教学培养了我解决复杂问题的综合能力）': '专业课解决问题能力',
    '请您评价本学年所修专业课在以下方面的情况（课程内容注重学科间的交叉与融合）': '专业课交叉融合',
    '请您评价本学年所修专业课在以下方面的情况（课程教学内容重视实践与理论的结合）': '专业课实践结合',
    '请您评价本学年所修专业课在以下方面的情况（需要非常努力才能达到课程要求）': '专业课努力程度',
    '请您评价本学年所修专业课在以下方面的情况（课程内容反映了学科发展的前沿内容）': '专业课前沿内容',
    # 教师教学方式
    '请您评价专业课任课教师的教学方式（课程以传统的教师讲授为主）': '传统讲授',
    '请您评价专业课任课教师的教学方式（课堂中加入互动环节，注重学生参与（提问、讨论））': '课堂互动',
    '请您评价专业课任课教师的教学方式（提供案例或实例讨论）': '案例讨论',
    '请您评价专业课任课教师的教学方式（采用小组合作的项目式教学方式）': '小组合作',
    # 课程考核方式
    '本学年，您参加过以下哪种课程考核方式（多选）（卷面考试）': '卷面考试',
    '本学年，您参加过以下哪种课程考核方式（多选）（随堂提问或测试）': '随堂测试',
    '本学年，您参加过以下哪种课程考核方式（多选）（课程论文或实验报告）': '论文报告',
    '本学年，您参加过以下哪种课程考核方式（多选）（课堂展示）': '课堂展示',
    '本学年，您参加过以下哪种课程考核方式（多选）（个人独立完成的课程作业）': '个人作业',
    '本学年，您参加过以下哪种课程考核方式（多选）（小组合作完成的课程作业）': '小组作业',
    '本学年，您参加过以下哪种课程考核方式（多选）（以上均没参加过）': '无考核方式',
    # 体育、美育、劳动教育满意度
    '请您评价本校的体育、美育、劳动教育的满意度（体育教育（如课内外体育活动、俱乐部等））': '体育教育满意度',
    '请您评价本校的体育、美育、劳动教育的满意度（美育教育（如公共艺术课程、艺术节、美育专题讲座等））': '美育教育满意度',
    '请您评价本校的体育、美育、劳动教育的满意度（劳动教育（如劳动类课程、勤工助学、社会公益活动等））': '劳动教育满意度',
    # 校园活动参与
    '本学年，您是否参加过以下活动（多选）（学生社团活动）': '参与社团活动',
    '本学年，您是否参加过以下活动（多选）（校园文化活动（如文化节、辩论赛、运动会等））': '参与校园文化',
    '本学年，您是否参加过以下活动（多选）（创新创业活动（如创新创业课程、创新创业活动和竞赛等））': '参与创新创业',
    '本学年，您是否参加过以下活动（多选）（国际交流活动（如海外访学、国际科研项目、参加国际会议、与来华交流的海外访学团交流等））': '参与国际交流',
    '本学年，您是否参加过以下活动（多选）（社会实践活动（如本校组织的志愿服务、参观考察等））': '参与社会实践',
    '本学年，您是否参加过以下活动（多选）（以上均未参与）': '未参与活动',
    # 各类活动满意度
    '请您评价学生社团活动的满意度（请您评价学生社团活动的满意度）': '社团活动满意度',
    '请您评价校园文化活动（如文化节、辩论赛、运动会等）的满意度（请您评价校园文化活动（如文化节、辩论赛、运动会等）的满意度）': '校园文化满意度',
    '请您评价创新创业活动（如创新创业课程、创新创业活动和竞赛等）的满意度（请您评价创新创业活动（如创新创业课程、创新创业活动和竞赛等）的满意度）': '创新创业满意度',
    '请您评价国际交流活动（如海外访学、国际科研项目、参加国际会议、与来华交流的海外访学团交流等）的满意度（请您评价国际交流活动（如海外访学、国际科研项目、参加国际会议、与来华交流的海外访学团交流等）的满意度）': '国际交流满意度',
    '请您评价社会实践活动（如本校组织的志愿服务、参观考察等）的满意度（请您评价社会实践活动（如本校组织的志愿服务、参观考察等）的满意度）': '社会实践满意度',
    # 实习实践满意度
    '请您评价本校实习实践各方面的满意度（实习实践的内容）': '实习内容满意度',
    '请您评价本校实习实践各方面的满意度（实习实践时间安排）': '实习时间满意度',
    '请您评价本校实习实践各方面的满意度（实习实践场地及设备）': '实习场地满意度',
    '请您评价本校实习实践各方面的满意度（实习指导情况）': '实习指导满意度',
    # 学习收获评价
    '您对在校学习收获的评价（分析问题、解决问题能力提升）': '问题解决能力提升',
    '您对在校学习收获的评价（自主学习能力提升）': '自主学习能力提升',
    '您对在校学习收获的评价（合作能力提升）': '合作能力提升',
    '您对在校学习收获的评价（书面表达和沟通能力提升）': '表达沟通能力提升',
    '您对在校学习收获的评价（有能力规划未来工作生活）': '未来规划能力提升',
    # 核心素养提升
    '在核心素养提升方面，您的满意度是（人文底蕴（人文积淀、人文情怀、审美情趣等））': '人文底蕴提升',
    '在核心素养提升方面，您的满意度是（科学精神（理性思维、批判质疑、勇于探究等））': '科学精神提升',
    '在核心素养提升方面，您的满意度是（学会学习（乐学善学、勤于反思、信息意识等））': '学会学习提升',
    '在核心素养提升方面，您的满意度是（健康生活（珍爱生命、健全人格、自我管理等））': '健康生活提升',
    '在核心素养提升方面，您的满意度是（责任担当（社会责任、国家认同、国际理解等））': '责任担当提升',
    '在核心素养提升方面，您的满意度是（实践创新（劳动意识、问题解决、技术应用等））': '实践创新提升',
    '在核心素养提升方面，您的满意度是（自我提升（变得自信、善于交友、学会包容等））': '自我提升',
    # 教师评价
    '您对本学年本校教师在以下方面的满意度（依法依规，履行教师职责）': '教师履职满意度',
    '您对本学年本校教师在以下方面的满意度（关心爱护学生）': '关爱学生满意度',
    '您对本学年本校教师在以下方面的满意度（师德师风及精神面貌）': '师德师风满意度',
    '您对本学年本校教师在以下方面的满意度（教学投入）': '教学投入满意度',
    '您对本学年本校教师在以下方面的满意度（总体教学满意度）': '教师总体满意度',
    # 教师教学行为评价
    '请评价本学年您的任课教师的教学行为（清楚地向学生解释了所学课程的目标和要求）': '课程目标解释',
    '请评价本学年您的任课教师的教学行为（课堂上激发学生的学习兴趣）': '激发学习兴趣',
    '请评价本学年您的任课教师的教学行为（课后为学生提供辅导答疑）': '课后辅导答疑',
    '请评价本学年您的任课教师的教学行为（重视立德树人，注重发挥课程育人的作用）': '立德树人',
    '请评价本学年您的任课教师的教学行为（教学方法灵活多样，引导学生进行创造性思考）': '创造性思考',
    # 学校服务评价
    '请您评价本校服务及教育教学是否符合以下情况（学生服务能够做到“一站式”的服务与解决，体现了对学生的关怀）': '一站式服务',
    '请您评价本校服务及教育教学是否符合以下情况（重视实验实训的安全教育与管理）': '实训安全管理',
    '请您评价本校服务及教育教学是否符合以下情况（任课教师积极参与学生活动）': '教师参与活动',
    '请您评价本校服务及教育教学是否符合以下情况（校内学术讲座报告活动多）': '学术讲座多',
    # 学生工作满意度
    '请您评价本校的学生工作的满意度（心理健康教育/咨询）': '心理健康满意度',
    '请您评价本校的学生工作的满意度（职业生涯规划和就业创业指导）': '职业规划满意度',
    '请您评价本校的学生工作的满意度（班主任、辅导员工作）': '班主任工作满意度',
    '请您评价本校的学生工作的满意度（学业指导工作）': '学业指导满意度',
    '请您评价本校的学生工作的满意度（重视家庭经济困难学生资助工作）': '资助工作满意度',
    # 学业指导服务
    '您是否接受过下述学业指导服务（学校或学院提供的辅导课程或相关活动）': '接受辅导课程',
    '您是否接受过下述学业指导服务（学业指导网站相关服务）': '接受网站指导',
    '您是否接受过下述学业指导服务（任课教师帮助解决相关问题）': '接受教师帮助',
    '您是否接受过下述学业指导服务（本科生导师帮助解决相关问题）': '接受导师帮助',
    '您是否接受过下述学业指导服务（辅导员帮助解决相关问题）': '接受辅导员帮助',
    '您是否接受过下述学业指导服务（没有接受过学业指导服务）': '未接受指导',
    # 资源条件满意度
    '请您评价本校资源条件的满意度（教室及教学设备）': '教室设备满意度',
    '请您评价本校资源条件的满意度（实验实训室条件）': '实训室满意度',
    '请您评价本校资源条件的满意度（图书馆资源）': '图书馆满意度',
    '请您评价本校资源条件的满意度（网络资源）': '网络资源满意度',
    '请您评价本校资源条件的满意度（体育文化等基础设施）': '体育设施满意度',
    '请您评价本校资源条件的满意度（住宿条件）': '住宿条件满意度',
    # 学校整体满意度
    '您对学校的整体满意度（您对学校的整体满意度）': '学校整体满意度',
}

# 应用列名映射
df_cleaned = df.rename(columns=column_mapping)

# 检查新列名
print("清洗后的列名数量:", len(df_cleaned.columns))
for i, col in enumerate(df_cleaned.columns[:]):
    print(f"{i+1:2d}. {col}")

### 3.2 处理缺失值

In [ ]:
# 检查缺失值情况
missing_values = df_cleaned.isnull().sum()
missing_percent = (missing_values / len(df_cleaned)) * 100

missing_df = pd.DataFrame({
    '缺失数量': missing_values,
    '缺失百分比': missing_percent
})

missing_df = missing_df[missing_df['缺失数量'] > 0].sort_values('缺失数量', ascending=False)
print("缺失值统计（按数量排序）:")
print(missing_df)

### 难受的是这个专业的缺失是因为前面学院的地方出现了某种奇怪的东西（XX方向、中外合办），然后专业留空，这些数据根本不符合大部分数据遵从的逻辑
### 所以还是直接先给这些加一个占位符，等待后续模型训练时再做特别处理


In [ ]:
df_cleaned['专业'] = df_cleaned['专业'].fillna("未指定")

print(df_cleaned[df_cleaned['专业'] == '未指定'])

### 3.4 处理异常值

In [ ]:
# 检查数值型列的基本统计信息
numeric_columns = df_cleaned.select_dtypes(include=[np.number]).columns
print("数值型列的基本统计信息:")
display(df_cleaned[numeric_columns].describe())

### 可以看到并没有什么夸张异常的地方。加之之前已经将满意度缩放到了1-0的尺度，我认为没有再处理的必要了

## 4. 清洗后数据概览

In [ ]:
# 显示清洗后的数据基本信息
print("清洗后的数据集形状:", df_cleaned.shape)
print("\n清洗后的数据集信息:")
print(df_cleaned.info())

# 显示清洗后的前几行
print("\n清洗后的数据集前5行:")
display(df_cleaned.head())

## 5. 保存清洗后的数据

In [ ]:
# 保存清洗后的数据
output_path = '../../data/intermediate/初步清洗_比赛数据.csv'
df_cleaned.to_csv(output_path, index=False, encoding='utf-8')
print(f"清洗后的数据已保存到: {output_path}")

# 同时保存为pickle格式，保留数据类型
pickle_path = '../../data/intermediate/cleaned_比赛数据.pkl'
df_cleaned.to_pickle(pickle_path)
print(f"清洗后的数据已保存为pickle格式: {pickle_path}")